# Applications of Artificial Intelligence
## Large Data Sets
### Indexing
In the previous demo you saw we could go through a large dataset in chunks. This time we'll show how we can leverage some of the material you learned last week about databases to work with various subsets of a large dataset efficiently.

We'll also introduce a new library on top of SQLite called SQLAlchemy. SQLAlchemy's primary focus is providing a layer of abstraction that allows you to write classes which can be stored in and retrieved from databases directly, without having to write SQL. But it is also the main way to interface with a database from Pandas, especially if you want to use Pandas with a database other than SQLite.

SQLAlchemy provides a really helpful interface between SQLite, which manages the database itself, and Pandas, which is obviously useful for doing data analysis within a single application, and actually reading CSV files in chunks. SQLAlchemy should be installed by default if you installed Python via Anaconda.

In [11]:
import pandas as pd
import sqlalchemy

database = sqlalchemy.create_engine('sqlite:///database.db')

The code above creates the SQLite database via SQLAlchemy, or connects if it already exists.

Now, to populate the database, we are going to read the CSV file in chunks in the same way that we did previously, inserting the data from those chunks to the database. This can be quite slow, but only needs to be done once on a large dataset. Note that we have to rename any columns from the CSV file that contain spaces, since this is not supported in the database.

*Note: the code below may take a couple of minutes to run.*

In [8]:
import time

start_time = time.process_time()

for df in pd.read_csv('data.csv', iterator=True, chunksize=100000):
    df = df.rename(columns={c: c.replace(' ', '_') for c in df.columns})
    df.to_sql('Sales', database, if_exists='append')
    
end_time = time.process_time()
print(f"Total time: {end_time-start_time:.1f} seconds")

Total time: 39.4 seconds


Pandas automatically set up and executed the SQL for us to store all of this data in the SQLite database. 

If we're curious, we can query the metadata of this table using a PRAGMA statement, to see the names of the columns, their types, and so on. This is quite a niche thing to want to do, and in reality you would probably just continue using this data via Pandas. But, just to show you what it has set up for us, you can see the result below. Notice it has had a good guess at the types of each of the columns.

(You can read more about this kind of meta-SQL [here](https://www.sqlitetutorial.net/sqlite-tutorial/sqlite-describe-table/).)

In [12]:
result = database.execute("PRAGMA TABLE_INFO('Sales');")
for row in result:
    print(", ".join(f"{key}: {item}" for key, item in row.items()))

AttributeError: 'Engine' object has no attribute 'execute'

We can manually run SELECT queries against the underlying SQLite database too. We could pick out some data in the middle of the original dataset (with index over 1 million) to confirm that it seems to be loading correctly, as shown in the cell below.

We can do this because Pandas automatically inserted its own row indices as a column called `index` into our table. Since this gives each row a unique value, it would be a good candidate for a primary key if we needed one (although it did not set this up). `index` is actually a bad name for a column because it already exists as a keyword in SQL, so to query it, we have to put square brackets around the name `[index]`. We also use LIMIT to just pull a few rows.

In [ ]:
result = database.execute("""SELECT [index], Region 
                             FROM Sales 
                             WHERE [index] > 1000000 
                             LIMIT 5;""")

for row in result:
    print(", ".join(f"{key}: {item}" for key, item in row.items()))

But if we want to do some data analysis on a subset of this data, then we will probably want the result back in a Pandas dataframe. We can pass the SQL query straight into Pandas to do this.

In [ ]:
start_time = time.process_time()

df = pd.read_sql_query('SELECT * FROM Sales WHERE Country="Oman";', database)

end_time = time.process_time()

print(f"Total time: {end_time-start_time:.1f} seconds")

In [ ]:
# preview all rows that have country Oman
print(f"Number of sales in Oman: {df.shape[0]}")
print("Preview of first three rows:")
df.head(3)

Just by using SQLite we are able to query this massive dataset in a reasonable amount of time. Database management systems do a lot of work by default to try to make queries fast, which may include some amount of indexing.

But if we know we're going to be doing lots of queries that include the Country column, we can tell the database to specifically create an index on this column.

In [ ]:
database.execute("CREATE INDEX idx_country ON Sales (Country);");

Now we can try the query again that pulls all the data from Oman.

In [ ]:
start_time = time.process_time()

df = pd.read_sql_query('SELECT * FROM Sales WHERE Country="Oman";', database)

end_time = time.process_time()

print(f"Total time: {end_time-start_time:.1f} seconds")

In my test, the time went from 0.8 to 0.2 seconds, cutting the query time to 25% of its original. This could make a big difference in a data intensive operation.